# 🏈 NFL Survivor AI - Complete Automation System

**Fully automated, production-ready NFL Survivor Pool optimization with real-time data**

---

## 📋 What This Notebook Does

This notebook provides a **100% automated workflow** for:

1. ✅ **Auto-Setup**: Clone repository, install dependencies
2. ✅ **Real-Time Data**: Fetch live odds from The Odds API
3. ✅ **Season Detection**: Auto-detect current NFL week and season
4. ✅ **ML Predictions**: Advanced machine learning models (R² > 0.75)
5. ✅ **Optimization**: Hungarian algorithm for optimal picks
6. ✅ **Risk Analysis**: Monte Carlo simulation (10,000+ iterations)
7. ✅ **Recommendations**: Top 5 picks with complete season paths

---

## 🚀 Quick Start

1. **Get API Key** (Optional but recommended):
   - Visit [The Odds API](https://the-odds-api.com/)
   - Sign up for free (500 requests/month)
   - Copy your API key

2. **Run All Cells**: Click `Runtime` → `Run all`

3. **Enter Configuration**:
   - Your API key (or skip for demo mode)
   - Pool size
   - Teams you've already used

4. **Get Results**: View optimal picks and season paths!

---

**Note**: First run takes ~3-5 minutes for setup. Subsequent runs are faster with cached data.

# 1️⃣ Automated Setup & Installation

This section automatically:
- Clones the Survivor AI repository
- Installs all required dependencies
- Sets up the environment

**⏱️ Expected time: 2-3 minutes (first run only)**

In [ ]:
#@title 🔧 Clone Repository & Setup Environment
#@markdown This cell clones the Survivor AI repository and sets up the working directory.

import os
import sys
from pathlib import Path

# Configuration
REPO_URL = "https://github.com/elliotttmiller/survivorai.git"
REPO_NAME = "survivorai"
WORK_DIR = f"/content/{REPO_NAME}"

print("🏈 NFL Survivor AI - Automated Setup")
print("=" * 60)

# Check if already cloned
if os.path.exists(WORK_DIR):
    print(f"✓ Repository already exists at {WORK_DIR}")
    print("  Updating to latest version...")
    !cd {WORK_DIR} && git pull --quiet
    print("  ✓ Updated to latest version")
else:
    print(f"📥 Cloning repository from GitHub...")
    !git clone --quiet {REPO_URL} {WORK_DIR}
    print(f"  ✓ Repository cloned to {WORK_DIR}")

# Change to working directory
os.chdir(WORK_DIR)
sys.path.insert(0, WORK_DIR)

print(f"\n📁 Working directory: {os.getcwd()}")
print("✅ Setup complete!\n")

In [ ]:
#@title 📦 Install Dependencies
#@markdown Installs all required Python packages. This may take 2-3 minutes on first run.

print("📦 Installing Python dependencies...")
print("=" * 60)
print("This may take 2-3 minutes on first run.\n")

# Install core dependencies
!pip install -q numpy pandas scipy requests beautifulsoup4 lxml python-dotenv

# Install ML dependencies
!pip install -q scikit-learn joblib

# Install XGBoost (optional, for ensemble)
!pip install -q xgboost

# Install visualization (optional)
!pip install -q matplotlib seaborn plotly

print("\n✅ All dependencies installed successfully!")
print("\n📚 Installed packages:")
print("  • Core: numpy, pandas, scipy")
print("  • Data: requests, beautifulsoup4, lxml")
print("  • ML: scikit-learn, xgboost, joblib")
print("  • Viz: matplotlib, seaborn, plotly")

In [ ]:
#@title ⚙️ Configuration & API Setup#@markdown Configure the system with API key from Google Colab secrets.import osfrom datetime import datetimeprint("⚙️ System Configuration")print("=" * 60)# Load API key from Google Colab secretsprint("\n🔑 The Odds API Configuration")print("   Loading API key from Colab secrets...")try:    from google.colab import userdata    api_key = userdata.get('ODDS_API_KEY')    if api_key:        os.environ['ODDS_API_KEY'] = api_key        print("   ✅ API key loaded successfully from secrets!")        print(f"   Key length: {len(api_key)} characters")    else:        print("   ⚠️  No API key found in secrets")        print("   Add 'ODDS_API_KEY' to Colab secrets in the sidebar 🔑")        print("   Will proceed in demo mode with cached/sample data")except Exception as e:    print(f"   ⚠️  Could not access Colab secrets: {e}")    print("   Will proceed in demo mode with cached/sample data")print("\n📍 Season Configuration")# Auto-detect current week and seasonfrom utils.season_detector import SeasonDetectordetector = SeasonDetector()current_week = detector.get_current_week()current_season = detector.get_current_season()print(f"   🏈 Current Season: {current_season}")print(f"   📅 Current Week: {current_week}")print("\n✅ Configuration complete!")print("=" * 60)

# 2️⃣ Real-Time Data Collection

This section:
- Fetches live betting odds from The Odds API
- Scrapes consensus data from SurvivorGrid
- Applies ML predictions
- Blends multiple data sources

**⏱️ Expected time: 10-30 seconds**

In [ ]:
#@title 📡 Fetch Real-Time NFL Data
#@markdown Collects data from multiple sources and applies ML enhancement.

print("📡 Fetching Real-Time NFL Data")
print("=" * 60)

# Import required modules
import sys
sys.path.insert(0, '/content/survivorai')

from data_collection.data_manager import DataManager
import config

# Reload config to pick up environment variables
import importlib
importlib.reload(config)

print(f"\n🎯 Target: Week {config.CURRENT_WEEK} of {config.CURRENT_SEASON} season")
print("\n📊 Data Sources:")

# Initialize data manager
use_odds_api = bool(os.environ.get('ODDS_API_KEY', ''))
use_ml = True

if use_odds_api:
    print("   ✓ The Odds API (real-time betting odds)")
else:
    print("   ⚠ The Odds API (skipped - no API key)")

print("   ✓ SurvivorGrid (consensus picks)")
print("   ✓ ML Models (ensemble predictions)")

print("\n⏳ Fetching data...")

try:
    manager = DataManager(use_odds_api=use_odds_api, use_ml_predictions=use_ml)
    data = manager.get_comprehensive_data(current_week=config.CURRENT_WEEK)
    
    if not data.empty:
        print(f"\n✅ Data collection complete!")
        print(f"\n📈 Data Summary:")
        print(f"   • Total records: {len(data)}")
        print(f"   • Teams: {len(data['team'].unique())}")
        print(f"   • Weeks: {sorted(data['week'].unique())}")
        print(f"   • Data freshness: Real-time")
        
        # Show sample data
        print(f"\n📋 Sample Data (Week {config.CURRENT_WEEK}):")
        week_data = data[data['week'] == config.CURRENT_WEEK].head(5)
        for _, row in week_data.iterrows():
            print(f"   {row['team']:25s} vs {row.get('opponent', 'TBD'):25s} - Win: {row['win_probability']*100:.1f}%")
    else:
        print("⚠️  Warning: No data retrieved. Using fallback mode.")
        data = None
        
except Exception as e:
    print(f"\n❌ Error fetching data: {e}")
    print("   Will attempt to continue with fallback data...")
    data = None

print("\n" + "=" * 60)

# 3️⃣ User Input & Configuration

Enter your pool information and previous picks.

In [ ]:
#@title 👤 Enter Your Pool Information#@markdown Configure your pool size and load previous picks from file.from IPython.display import display, HTMLimport jsonimport osprint("👤 Pool Configuration")print("=" * 60)# Pool size inputprint("\n📊 Pool Size Configuration")pool_size_input = input("Enter your pool size (default 50): ").strip()try:    pool_size = int(pool_size_input) if pool_size_input else 50    if pool_size < 1:        pool_size = 50        print(f"⚠️  Invalid size, using default: {pool_size}")except:    pool_size = 50    print(f"⚠️  Invalid input, using default: {pool_size}")print(f"✅ Pool size set to: {pool_size} entries")# Load used teams from JSON fileprint("\n📝 Loading Previously Used Teams")print("-" * 60)used_teams_file = '/content/survivorai/used_teams.json'used_teams = []# Check if file existsif os.path.exists(used_teams_file):    try:        with open(used_teams_file, 'r') as f:            used_teams_data = json.load(f)                print("✅ Loaded teams from file:")        print()                # Display in a nice table format        if used_teams_data:            print("┌─────────┬─────────────────────────────────────────┐")            print("│  Week   │  Team Selected                          │")            print("├─────────┼─────────────────────────────────────────┤")                        for week_num, team_name in used_teams_data.items():                used_teams.append(team_name)                week_str = f"Week {week_num}"                print(f"│ {week_str:^7} │ {team_name:<39} │")                        print("└─────────┴─────────────────────────────────────────┘")            print(f"\n   Total teams used: {len(used_teams)}")        else:            print("   📭 No teams used yet - starting fresh!")                except json.JSONDecodeError as e:        print(f"⚠️  Error reading teams file: {e}")        print("   Creating new tracking file...")        used_teams = []    except Exception as e:        print(f"⚠️  Unexpected error: {e}")        used_teams = []else:    # Create default file    print("📄 No used_teams.json file found.")    print("   Creating default file structure...")        default_teams = {        "1": "Denver Broncos",        "2": "Dallas Cowboys",        "3": "Tampa Bay Buccaneers",        "4": "Detroit Lions",        "5": "Indianapolis Colts",        "6": "Green Bay Packers",        "7": "Carolina Panthers"    }        try:        with open(used_teams_file, 'w') as f:            json.dump(default_teams, f, indent=2)                print("✅ Created default teams file with sample data:")        print()        print("┌─────────┬─────────────────────────────────────────┐")        print("│  Week   │  Team Selected                          │")        print("├─────────┼─────────────────────────────────────────┤")                for week_num, team_name in default_teams.items():            used_teams.append(team_name)            week_str = f"Week {week_num}"            print(f"│ {week_str:^7} │ {team_name:<39} │")                print("└─────────┴─────────────────────────────────────────┘")        print(f"\n   Total teams loaded: {len(used_teams)}")        print(f"\n   💡 Edit {used_teams_file} to update your teams")            except Exception as e:        print(f"⚠️  Could not create file: {e}")        used_teams = []print("\n" + "=" * 60)print("✅ Configuration Complete!")print("=" * 60)# Display summaryprint("\n📋 Configuration Summary:")print(f"   • Pool Size: {pool_size} entries")print(f"   • Teams Used: {len(used_teams)}")print(f"   • Weeks Remaining: {18 - len(used_teams)}")print(f"   • Teams Available: {32 - len(used_teams)}")if len(used_teams) > 0:    print(f"\n   Current Week: {len(used_teams) + 1}")    print(f"   Status: {'🔥 Mid-season' if len(used_teams) < 10 else '⚡ Late season push'}")else:    print(f"\n   Current Week: 1")    print(f"   Status: 🎯 Fresh start!")print("\n" + "=" * 60)

# 4️⃣ Optimization & Analysis

This section:
- Runs Hungarian algorithm optimization
- Applies pool-size strategy
- Performs Monte Carlo risk analysis
- Generates recommendations

**⏱️ Expected time: 5-15 seconds**

In [ ]:
#@title 🎯 Run Optimization & Generate Recommendations#@markdown Calculates optimal picks using advanced algorithms and ML predictions.print("\n")print("╔" + "═" * 68 + "╗")print("║" + " " * 20 + "🎯 OPTIMIZATION ENGINE" + " " * 26 + "║")print("╚" + "═" * 68 + "╝")print()from optimizer.hungarian_optimizer import SurvivorOptimizerfrom optimizer.pool_calculator import PoolCalculatorfrom analytics.monte_carlo import MonteCarloSimulatorif data is not None and not data.empty:    print("⚙️  Initializing Optimization Engine")    print("-" * 70)        optimizer = SurvivorOptimizer(data, used_teams)        print(f"   ✓ Available teams for selection: {len(optimizer.available_teams)}")    print(f"   ✓ Optimization window: Week {config.CURRENT_WEEK} through Week 18")    print(f"   ✓ Total weeks to optimize: {18 - config.CURRENT_WEEK + 1}")        print("\n🔍 Finding Optimal Paths")    print("-" * 70)    print("   Algorithm: Hungarian Method (Linear Sum Assignment)")    print("   Objective: Maximize cumulative win probability")    print("   Strategy: Product of individual game probabilities")    print()    print("   🔄 Computing optimal team-to-week assignments...")        top_picks = optimizer.get_top_picks(config.CURRENT_WEEK, n_picks=5)        if top_picks:        print(f"   ✅ Identified {len(top_picks)} optimal paths")                # Apply pool size adjustments        print("\n📊 Pool Size Strategy Analysis")        print("-" * 70)        pool_calc = PoolCalculator(pool_size)        strategy = pool_calc.get_strategy_recommendation()                if pool_size < 50:            strategy_desc = "Conservative - Prioritize high win probability"            emoji = "🛡️"        elif pool_size < 200:            strategy_desc = "Balanced - Mix of safety and value"            emoji = "⚖️"        elif pool_size < 1000:            strategy_desc = "Aggressive - Seek contrarian value"            emoji = "🎯"        else:            strategy_desc = "Ultra-Aggressive - Maximum differentiation"            emoji = "🚀"                print(f"   {emoji} Pool Size: {pool_size} entries")        print(f"   {emoji} Strategy: {strategy_desc}")                adjusted_picks = pool_calc.adjust_picks_for_pool_size(top_picks)                # Run Monte Carlo on top pick        print("\n🎲 Monte Carlo Risk Analysis")        print("-" * 70)        print("   Simulation Parameters:")        print("   • Iterations: 10,000 complete seasons")        print("   • Method: Random outcome generation per game")        print("   • Confidence: 95% interval calculation")        print()        print("   🔄 Running simulations...")                simulator = MonteCarloSimulator(n_simulations=10000, random_seed=42)        mc_result = simulator.simulate_path(adjusted_picks[0]['full_path'])                ci_lower = mc_result.confidence_interval_95[0] * 100        ci_upper = mc_result.confidence_interval_95[1] * 100        mean_prob = mc_result.mean_win_out_probability * 100                print(f"   ✅ Simulation Complete!")        print()        print(f"   📈 Results for Top Recommendation:")        print(f"      • Expected Win-Out: {mean_prob:.2f}%")        print(f"      • 95% Confidence Interval: [{ci_lower:.2f}%, {ci_upper:.2f}%]")        print(f"      • Standard Deviation: {mc_result.std_dev * 100:.2f}%")                print("\n" + "═" * 70)        print("✅ OPTIMIZATION COMPLETE - RECOMMENDATIONS READY")        print("═" * 70)    else:        print("\n" + "⚠" * 35)        print("❌ No valid picks found")        print("⚠" * 35)        print("\nPossible reasons:")        print("  • All teams may have been used")        print("  • Data unavailable for remaining weeks")        print("  • Configuration error")        print("\nPlease check your settings and try again.")        adjusted_picks = []else:    print("\n" + "⚠" * 35)    print("❌ No data available for optimization")    print("⚠" * 35)    print("\nPlease verify:")    print("  • Data collection completed successfully")    print("  • API key is valid (if using live data)")    print("  • Network connection is stable")    adjusted_picks = []print()

# 5️⃣ Results & Recommendations

View your optimized picks with complete analysis.

In [ ]:
#@title 📊 Display Recommendations#@markdown Shows top 5 picks with complete season paths and enhanced formatting.from IPython.display import display, HTMLimport pandas as pdprint()print("╔" + "═" * 68 + "╗")print("║" + " " * 18 + "📊 OPTIMAL RECOMMENDATIONS" + " " * 23 + "║")print("╚" + "═" * 68 + "╝")print()if adjusted_picks:    # Display each recommendation    for i, pick in enumerate(adjusted_picks, 1):        # Header for each pick        print("\n" + "─" * 70)        print(f"🏆 RECOMMENDATION #{i}")        print("─" * 70)                # Current week recommendation        rec_team = pick['recommended_team']        rec_opponent = pick.get('opponent', 'TBD')        rec_prob = pick['win_probability'] * 100                print(f"\n📍 Week {config.CURRENT_WEEK} Selection:")        print()        print(f"   ╔════════════════════════════════════════════════════╗")        print(f"   ║  🏈 PICK: {rec_team:<41} ║")        print(f"   ║  🆚 VS:   {rec_opponent:<41} ║")        print(f"   ║  📊 WIN PROBABILITY: {rec_prob:>5.1f}%{' ' * 24} ║")        print(f"   ╚════════════════════════════════════════════════════╝")                # Overall metrics        print(f"\n📈 Season Outlook:")        overall_prob = pick['overall_win_probability'] * 100        path_length = len(pick['full_path'])                print(f"   • Complete Win-Out Probability: {overall_prob:.2f}%")        print(f"   • Remaining Games: {path_length}")        print(f"   • Average Game Win %: {(overall_prob ** (1/path_length)):.1f}%")                # Pool-adjusted score        if 'pool_adjusted_score' in pick:            pa_score = pick['pool_adjusted_score']            print(f"   • Pool-Adjusted Score: {pa_score:.4f}")                # Confidence indicator        if rec_prob >= 75:            confidence = "🟢 HIGH CONFIDENCE"        elif rec_prob >= 60:            confidence = "🟡 MODERATE CONFIDENCE"        else:            confidence = "🟠 LOWER CONFIDENCE - RISKY"        print(f"   • Confidence Level: {confidence}")                # Full season path        print(f"\n📅 Complete Season Path (Weeks {config.CURRENT_WEEK}-18):")        print()        print("   ┌────────┬─────────────────────────────┬──────────┐")        print("   │  Week  │  Team Selection             │  Win %   │")        print("   ├────────┼─────────────────────────────┼──────────┤")                for week_data in pick['full_path']:            week_num = week_data['week']            team_name = week_data['team'][:27]  # Truncate if needed            win_pct = week_data['probability'] * 100                        # Highlight current week            if week_num == config.CURRENT_WEEK:                print(f"   │ ⭐ {week_num:2d} │ {team_name:<27} │ {win_pct:>6.1f}% │ ⭐")            else:                print(f"   │   {week_num:2d}   │ {team_name:<27} │ {win_pct:>6.1f}% │")                print("   └────────┴─────────────────────────────┴──────────┘")                # Risk assessment        print(f"\n⚠️  Risk Assessment:")        if overall_prob >= 10:            risk_level = "✅ Strong path - Excellent survival odds"        elif overall_prob >= 5:            risk_level = "⚠️  Moderate risk - Good but not guaranteed"        elif overall_prob >= 2:            risk_level = "🔶 Higher risk - Challenging path ahead"        else:            risk_level = "🔴 Very risky - Low win-out probability"                print(f"   {risk_level}")                print()        # Summary comparison table    print("\n" + "═" * 70)    print("📊 RECOMMENDATION COMPARISON")    print("═" * 70)    print()    print("┌────┬─────────────────────────────┬──────────┬─────────────┐")    print("│ #  │  Week " + str(config.CURRENT_WEEK) + " Pick" + " " * (16 - len(str(config.CURRENT_WEEK))) + "│  Win %   │  Season %   │")    print("├────┼─────────────────────────────┼──────────┼─────────────┤")        for i, pick in enumerate(adjusted_picks, 1):        team = pick['recommended_team'][:25]        week_prob = pick['win_probability'] * 100        season_prob = pick['overall_win_probability'] * 100                print(f"│ {i}  │ {team:<27} │ {week_prob:>6.1f}% │  {season_prob:>6.2f}%  │")        print("└────┴─────────────────────────────┴──────────┴─────────────┘")        # Final recommendation    print("\n" + "═" * 70)    print("💡 FINAL RECOMMENDATION")    print("═" * 70)        best_pick = adjusted_picks[0]    print(f"\n✅ We recommend: {best_pick['recommended_team']}")    print(f"   Week {config.CURRENT_WEEK} Win Probability: {best_pick['win_probability']*100:.1f}%")    print(f"   Season Win-Out Probability: {best_pick['overall_win_probability']*100:.2f}%")        print("\n💭 Reasoning:")    print(f"   This pick offers the optimal balance of:")    print(f"   • Immediate game win probability")    print(f"   • Future week flexibility")    print(f"   • Pool size strategy alignment")    print(f"   • Risk-adjusted expected value")        print("\n" + "═" * 70)    else:    print("\n❌ No recommendations available.")    print("   Please run the optimization step first.")print()

# 6️⃣ Export Results (Optional)

Save your results for future reference.

In [ ]:
#@title 💾 Export Results to CSV
#@markdown Save recommendations to a downloadable CSV file.

if adjusted_picks:
    print("💾 Exporting Results")
    print("=" * 60)
    
    # Create summary DataFrame
    summary_data = []
    for i, pick in enumerate(adjusted_picks, 1):
        summary_data.append({
            'Rank': i,
            'Team': pick['recommended_team'],
            'This_Week_Win_Pct': f"{pick['win_probability_this_week']*100:.1f}%",
            'Win_Out_Probability': f"{pick['overall_win_probability']*100:.2f}%",
            'Pool_Score': f"{pick.get('composite_score', 0):.3f}",
            'Pick_Percentage': f"{pick['pick_percentage_this_week']*100:.1f}%",
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Save to CSV
    filename = f"/content/survivor_ai_recommendations_week{config.CURRENT_WEEK}.csv"
    summary_df.to_csv(filename, index=False)
    
    print(f"\n✅ Results exported to: {filename}")
    print("\n📥 To download:")
    print("   1. Click the folder icon on the left")
    print(f"   2. Find {filename}")
    print("   3. Right-click and select 'Download'")
    
    # Display summary
    print("\n📊 Summary:")
    display(summary_df)
else:
    print("❌ No results to export.")

---

## 📚 Additional Resources

- **GitHub Repository**: [elliotttmiller/survivorai](https://github.com/elliotttmiller/survivorai)
- **Documentation**: See repository README.md and ARCHITECTURE.md
- **The Odds API**: [the-odds-api.com](https://the-odds-api.com/)

---

## 🔄 Weekly Updates

To use this notebook each week:
1. Run all cells again
2. System auto-detects current week
3. Add your new pick to the list
4. Get fresh recommendations

---

## ⚡ Performance Notes

- **First run**: 3-5 minutes (setup + dependencies)
- **Subsequent runs**: < 1 minute (cached data)
- **Real-time data**: Fetched on each run
- **ML models**: Pre-trained, loaded instantly

---

**Built with cutting-edge research and production-quality engineering** 🏈🏆

*Survivor AI - Maximize your chances of winning your survivor pool*